In [1]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
def nan_to_null(f,
        _NULL=psycopg2.extensions.AsIs('NULL'),
        _Float=psycopg2.extensions.Float):
    if not np.isnan(f):
        return _Float(f)
    return _NULL

psycopg2.extensions.register_adapter(float, nan_to_null)

In [3]:
conn = psycopg2.connect(
    host=os.environ['POSTGRES_HOST'],
    port=os.environ['POSTGRES_PORT'],
    database=os.environ['OLAP_DB'],
    user=os.environ['POSTGRES_USER'],
    password=os.environ['POSTGRES_PASSWORD']
)

In [4]:
def execute_values(conn, df, table): 
  
    tuples = [tuple(x) for x in df.to_numpy()] 
  
    cols = ','.join(list(df.columns)) 
    # SQL query to execute 
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
    cursor = conn.cursor() 
    try: 
        extras.execute_values(cursor, query, tuples) 
        conn.commit() 
    except (Exception, psycopg2.DatabaseError) as error: 
        print("Error: %s" % error) 
        conn.rollback() 
        cursor.close() 
        return 1
    cursor.close() 

# Upload properties

In [6]:
final = pd.read_csv('../data/processed/20231116/processed.csv')
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48729 entries, 0 to 48728
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                48729 non-null  int64  
 1   name              48729 non-null  object 
 2   url               48729 non-null  object 
 3   price             48657 non-null  float64
 4   num_bedroom       48304 non-null  float64
 5   num_bathroom      48001 non-null  float64
 6   cost_psf          47955 non-null  float64
 7   address           37597 non-null  object 
 8   road_name         37547 non-null  object 
 9   building          30414 non-null  object 
 10  postal_code       33697 non-null  float64
 11  latitude          37597 non-null  float64
 12  longitude         37597 non-null  float64
 13  floor_area        48729 non-null  int64  
 14  land_area         9166 non-null   float64
 15  walk_destination  40209 non-null  object 
 16  walk_distance_m   40209 non-null  float6

In [7]:
columns = [
    'id','name','url','price','num_bedroom','num_bathroom','cost_psf',
    'address','road_name','building','postal_code','latitude','longitude',
    'floor_area','land_area','walk_destination','walk_distance_m','walk_time_mins',
    'lease_duration','completion','type',
]
properties = final[columns]

execute_values(conn, properties, 'properties')

In [10]:
cur = conn.cursor()
cur.execute('select count(*) from properties')
cur.fetchall()

[(48729,)]

# Upload Amenities

In [ ]:
! python ../pipeline_update_amenities/overall_pipeline.py

In [13]:
cur = conn.cursor()
cur.execute('select count(*) from properties_w_amenities')
cur.fetchall()

[(48729,)]